In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
def blog_to_podcast(blog, content_type):
    exclude = {"id", "page_ptr_id", "page_ptr", "translation_key"}
    kwargs = {
        f.name: getattr(blog, f.name)
        for f in Blog._meta.fields
        if f.name not in exclude
    }
    kwargs["slug"] = f"new_{blog.slug}"
    kwargs["content_type"] = content_type
    kwargs["new_itunes_artwork"] = blog.itunes_artwork
    kwargs["new_itunes_categories"] = blog.itunes_categories
    kwargs["new_keywords"] = blog.keywords
    kwargs["new_explicit"] = blog.explicit
    return Podcast(**kwargs)

<IPython.core.display.Javascript object>

In [3]:
# first migration to add podcast model
from django.core.management import call_command
call_command("migrate")

# get the original blog + parent
blog = Blog.objects.first()
original_slug = blog.slug
blog_parent = Page.objects.parent_of(blog).first()

# fix hostname and port
site = Site.objects.first()
site.hostname = "localhost"
site.port = 8000
site.save()

# create new page
podcast_content_type = ContentType.objects.get(app_label="cast", model="podcast")
podcast = blog_to_podcast(blog, podcast_content_type)
podcast = blog_parent.add_child(instance=podcast)

# fix treebeard, dunno why this is needed
from django.core.management import call_command
call_command("fixtree")
podcast = Podcast.objects.first()  # super important!

# move children - this is extremely brittle!
from wagtail.actions.move_page import MovePageAction
for child in blog.get_children():
    mpa = MovePageAction(child, podcast, pos="last-child")
    mpa.execute()

# delete old page
blog.delete()

# restore slug
podcast.slug = original_slug
podcast.save()

Operations to perform:
  Apply all migrations: account, admin, auth, authtoken, cast, contenttypes, django_comments, fluent_comments, indieweb, sessions, sites, socialaccount, taggit, threadedcomments, users, wagtailadmin, wagtailcore, wagtaildocs, wagtailembeds, wagtailforms, wagtailimages, wagtailredirects, wagtailsearch, wagtailusers
Running migrations:
  Applying cast.0033_add_new_podcast_model... OK
Checking page tree for problems...
Incorrect numchild value found for pages: [120]
All problems fixed.

Checking collection tree for problems...
No problems found.



<IPython.core.display.Javascript object>

In [4]:
episode = Episode.objects.first()

<IPython.core.display.Javascript object>

In [5]:
episode.podcast

AttributeError: 'Episode' object has no attribute 'podcast'

<IPython.core.display.Javascript object>

In [10]:
page = episode.get_parent()

<IPython.core.display.Javascript object>

In [11]:
page.specific

<Podcast: Python Podcast>

<IPython.core.display.Javascript object>

In [4]:
Blog.objects.first()

ProgrammingError: column cast_blog.itunes_artwork_id does not exist
LINE 1: ...st_blog"."email", "cast_blog"."comments_enabled", "cast_blog...
                                                             ^


<IPython.core.display.Javascript object>

In [1]:
Blog.objects.first()

ProgrammingError: column cast_blog.itunes_artwork_id does not exist
LINE 1: ...st_blog"."email", "cast_blog"."comments_enabled", "cast_blog...
                                                             ^


In [1]:
Podcast.objects.all()

NameError: name 'Podcast' is not defined

In [2]:
print("hello world")

hello world


In [1]:
from cast.models.pages import Podcast

ImportError: cannot import name 'Podcast' from 'cast.models.pages' (/Users/jochen/.virtualenvs/pp/lib/python3.11/site-packages/cast/models/pages.py)